In [2]:
import os
import re
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, rdFreeSASA

# Define directory 
aux_directory = "output_sol/scratch/chmbnn/mopac_jobs/eloise_clayton"

In [3]:
import os
import re
import pandas as pd
from rdkit import Chem
import rdkit.Chem.rdFreeSASA as rdFreeSASA

def parse_aux_file(file_path):
    with open(file_path, "r") as f:
        content = f.read()

    # Extract molecular weight using scientific notation handling
    mw_match = re.search(r"MOLECULAR_WEIGHT:AMU=\+([\d.]+)D([+-]?\d+)", content)
    mw = float(mw_match.group(1)) * (10 ** int(mw_match.group(2))) if mw_match else None

    # Extract empirical formula
    formula_match = re.search(r'EMPIRICAL_FORMULA="([^"]+)"', content)
    formula = formula_match.group(1) if formula_match else None

    # Extract atomic elements
    atoms_match = re.search(r'ATOM_EL\[\d+\]=\s+([\w\s]+)', content)
    atom_list = [atom.strip() for atom in atoms_match.group(1).split() if atom.strip().isalpha()] if atoms_match else []

    # Extract atomic coordinates
    coord_match = re.search(r'ATOM_X:ANGSTROMS\[\d+\]=([\s\d\-.]+)', content)
    coords = []
    if coord_match:
        for line in coord_match.group(1).strip().split("\n"):
            parts = line.strip().split()
            if len(parts) == 3:
                coords.append(list(map(float, parts)))
    
    return formula, mw, atom_list, coords

def compute_sasa(atom_list, coords):
    if len(atom_list) != len(coords):
        print("Warning: Mismatch between atoms and coordinates. Skipping SASA calculation.")
        return None

    mol = Chem.RWMol()
    for atom in atom_list:
        try:
            mol.AddAtom(Chem.Atom(atom))
        except:
            print(f"Skipping unknown atom: {atom}")
            return None

    conf = Chem.Conformer(len(atom_list))
    for i, (x, y, z) in enumerate(coords):
        conf.SetAtomPosition(i, (x, y, z))
    
    mol.AddConformer(conf)
    mol = mol.GetMol()

    # Compute SASA
    radii = rdFreeSASA.classifyAtoms(mol)
    sasa = rdFreeSASA.CalcSASA(mol, radii)
    
    return sasa

def extract_solute_solvent(filename):
    if filename.endswith('.aux'):
        parts = filename[:-4].split('_', 1)
        if len(parts) == 2:
            return parts[0], parts[1] 
    return None, None

# Process all .aux files
data = []
for file_name in os.listdir(aux_directory):
    if file_name.endswith(".aux"):
        file_path = os.path.join(aux_directory, file_name)
        solute_inchikey, solvent_name = extract_solute_solvent(file_name)
        
        formula, mw, atom_list, coords = parse_aux_file(file_path)
        sasa = compute_sasa(atom_list, coords) if atom_list and coords else None

        data.append([solute_inchikey, solvent_name, sasa, mw])

# Create DataFrame and save results
df = pd.DataFrame(data, columns=["solute_inchikey", "solvent_name", "SASA", "MW"])
df.to_csv("solute_sasa_mw_results.csv", index=False)

                  solute_inchikey     solvent_name        SASA       MW
0     ASWVTGNCAZCNNR-WYUMXYHSNA-N        1-octanol  348.614971  278.328
1     DFXQXFGFOLXAPO-KZFATGLANA-N      1,4-dioxane  190.732933  201.566
2     ISAOCJYIOMOJEB-UHFFFAOYNA-N  2-butoxyethanol  279.381632  212.248
3     PJANXHGTPQOBST-VAWYXSNFNA-N     ethylbenzene  252.896718  180.249
4     WPYMKLBDIGXBTP-FZOZFQFYNA-N         methanol  159.630691  122.123
...                           ...              ...         ...      ...
3546  NIHNNTQXNPWCJQ-UHFFFAOYNA-N          heptane  225.102388  166.222
3547  LVHBHZANLOWSRM-HJYFZBQUNA-N    ethyl_acetate  168.016964  130.100
3548  FFGPTBGBLSHEPO-ZHLVXTBQNA-N     acetonitrile  296.704121  236.273
3549  PJANXHGTPQOBST-VAWYXSNFNA-N        1-octanol  252.896718  180.249
3550  BBEAQIROQSPTKN-UHFFFAOYNA-N     acetophenone  241.464832  202.255

[3551 rows x 4 columns]
